In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My\ Drive/TRVQA

# List files to make sure we're in the expected directory.
# Your output will look different, showing your own Drive files here.
!ls

/content/drive/My Drive/TRVQA
bench_10S_10I		       bench_out_pl
bench_10S_10I_start	       bench_out_shot_diff
benchmark		       bench_out_shot_diff_2436
bench_out		       bench_out_shot_diff_24_real
bench_out_100		       bench_out_shot_diff_36_real
bench_out_1000		       bench_out_shot_diff_4816_real
bench_out_100_old	       bench_out_shot_diff_52
bench_out_150		       bench_out_shot_diff_52_real
bench_out_2436		       bench_tsp_4816
bench_out_4816		       bench_tsp_49
bench_out_4816_lr_3	       circuit.py
bench_out_4816_old	       ex_value_pots
bench_out_4816_sv_compare      gates
bench_out_52		       hamiltonian
bench_out_52_lr		       __init__.py
bench_out_52_memory	       maxcut_comparison_CPU
bench_out_52_old	       maxcut_comparison_GPU
bench_out_52_opt	       measure
bench_out_comparison	       optimization
bench_out_comparison_cpu_real  __pycache__
bench_out_comparison_gpu       utils


In [ ]:
import sys
sys.path.append('/content/drive/My\ Drive/TRVQA')


<>:2: SyntaxWarning: invalid escape sequence '\ '
<>:2: SyntaxWarning: invalid escape sequence '\ '
/tmp/ipython-input-691171030.py:2: SyntaxWarning: invalid escape sequence '\ '
  sys.path.append('/content/drive/My\ Drive/TRVQA')


In [ ]:
!pip install qiskit
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 40.2 MB/s eta 0:00:00


In [ ]:
import sys
import os
import sys
import os
# Add the parent directory of TRVQA to sys.path
from measure.enums import MeasureMethod
from optimization.gradients.vanilla_parameter_shift import vanilla_parameter_shift
from optimization.optimization import minimize, minimize_custom
#from utils.maxcut import gengraph, create_hamiltonian, make_hamiltonian
import torch
from torch.autograd import Function
from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit import QuantumCircuit

import numpy as np
import torch
from circuit import Circuit

def compute_maxcut_value(bitstring, graph):
    """Computes the MaxCut value for a given bitstring solution."""
    cut_value = sum(w for (u,v), w in graph if bitstring[u] != bitstring[v])
    return cut_value

# for s in best_value:
#     print(compute_maxcut_value(s, g))

from pyparsing import Optional
import torch
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Operator, SparsePauliOp
from optimization.optimization import minimize, minimize_custom
# TRVQA imports

from circuit import Circuit
from hamiltonian.hamiltonian import Hamiltonian
from measure.enums import MeasureMethod
torch.cuda.empty_cache()
import numpy as np
import torch
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Operator, SparsePauliOp
import torch

from measure.enums import MeasureMethod
from optimization.optimization import minimize
from utils.maxcut import gengraph, create_hamiltonian,make_hamiltonian
from hamiltonian.hamiltonian import Hamiltonian
import numpy as np
import torch
from circuit import Circuit
import time


In [ ]:
import gurobipy as gp
from gurobipy import GRB
import networkx as nx

def maxcut_value(G) -> float:
    """
    Solve weighted Max-Cut on an undirected NetworkX graph using Gurobi.

    Args:
        G: networkx.Graph
           Undirected graph. Edge weights read from attribute 'weight' (default 1.0).

    Returns:
        float: Optimal Max-Cut value.
    """
    # if not isinstance(G, nx.Graph) or G.is_directed():
    #     raise ValueError("G must be an undirected networkx.Graph")

    model = gp.Model("MaxCut")
    model.Params.OutputFlag = 0  # silence solver output

    # Binary variable x[i] ∈ {0,1} indicates partition side of node i
    #nodes = list(G.nodes())
    # Number of nodes
    nodes = set()
    for e,w in G:
        u,v = e
        nodes.add(u)
        nodes.add(v)
    n = len(nodes)

    x = model.addVars(nodes, vtype=GRB.BINARY, name="x")
    # Objective: Maximize the cut value
    model.setObjective(
        gp.quicksum(w * (x[u] + x[v] - 2 * x[u] * x[v]) for (u, v), w in G),
        GRB.MAXIMIZE
    )

    # Optimize the model
    model.optimize()


    if model.Status != GRB.OPTIMAL:
        raise RuntimeError(f"Gurobi did not find an optimal solution (status={model.Status}).")

    return float(model.ObjVal)
import json

# Load
with open("benchmark/graphs.json", "r") as f:
    loaded_graphs = json.load(f)

# Convert to tuples
loaded_graphs = [
    [((int(e[0][0]), int(e[0][1])), int(e[1])) for e in g]
    for g in loaded_graphs
]

print(loaded_graphs[0][:3])
N_list = [4, 8, 16, 24, 36, 52, 70, 90, 100]
from dataclasses import dataclass, asdict
from typing import Dict, List, Tuple, Callable, Any
import torch, numpy as np

# --- Result container ---------------------------------------------------------
@dataclass
class RunLog:
    exp_values: List[float]
    best_results: List[Any]   # bitstrings, ints, etc.
    iter_times: List[float]
    final_theta: torch.Tensor

# --- Helpers ------------------------------------------------------------------
def build_qiskit_like_hamiltonian(hm):
    """hm is your [(coeff, bool_list), ...]"""
    pauli_strings, coefficients = [], []
    for coeff, bool_list in hm:
        pauli_strings.append(''.join('Z' if b else 'I' for b in bool_list))
        coefficients.append(coeff)
    return Hamiltonian(len(bool_list), pauli_strings, coefficients)

def build_circuit(N: int, D: int, rank: int = 10, device: str = 'cuda', isRing: bool = True) -> Circuit:
    c = Circuit(N, device=device)
    for _ in range(D):
        for j in range(N): c.rx(j)
        for j in range(N): c.rz(j)
        for j in range(N-1): c.cx(j, j+1)
        if isRing:
            c.cx(N-1, 0)
    c.rank = rank
    return c

# --- Method runners -----------------------------------------------------------
def run_statevector_vqe(N: int, D: int, h, iter_steps: int, lr: float = 1e-2, theta0=None, isRing=True) -> RunLog:
    model = QuantumCircuitTorch(theta0, N, D, h, isRing=isRing)
    opt = torch.optim.AdamW(model.parameters(), lr=lr)

    exp_vals, best_results, iter_times = [], [], []
    for _ in range(iter_steps):
        opt.zero_grad()
        out = model()                   # expectation value (loss)
        out.backward()
        opt.step()
        exp_vals.append(float(out.item()))
        best_results.append(get_opt_cut_sv(model.theta, N, D, h, isRing))
        # if you track iteration time, append here (e.g., time.perf_counter deltas)
    return RunLog(exp_values=exp_vals, best_results=best_results, iter_times=iter_times, final_theta=model.theta.detach().clone())

def run_tr_vqe(method, circuit, theta0, hamil, iter_steps: int, lr: float, grad_kwargs: dict) -> RunLog:
    """Generic TR-VQE using your Optimizer + ParameterShift flavors."""
    # # Build gradient object with provided kwargs, override measure method:
    from optimization.gradients.batch_parameter_shift import BatchParameterShiftGradient
    from optimization.optimizer import Optimizer
    grad = BatchParameterShiftGradient(
        torch.pi/2,
        grad_kwargs.get("batches", 100),
        grad_kwargs.get("shots", int(1e3)),
        method,                      # MeasureMethod
        grad_kwargs.get("D", 1)
    )
    opt = Optimizer(torch.optim.AdamW, {'lr': lr})
    theta_init = theta0.clone().detach().requires_grad_(True)
    new_theta, exp_vals, best_results, iter_times = minimize(
        circuit, theta_init, hamil, opt, grad, iter_steps, best_value_method="argmax_tr_noinv_LE"
    )
    return RunLog(
        exp_values=[float(v.real) for v in exp_vals],
        best_results=list(best_results),
        iter_times=list(iter_times),
        final_theta=new_theta.detach().clone()
    )


[((0, 3), 6), ((0, 1), 4), ((0, 2), 5)]


In [ ]:
import torch
from torch.autograd import Function
from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit import QuantumCircuit

def run_circuit_sv(theta,N,D,h, isRing):
    c = QuantumCircuit(N)
    theta_t = 0
    for i in range(D):
        for ind1 in range(N):
            c.rx(theta[theta_t].detach().cpu().item(), ind1)
            theta_t+=1
        for ind1 in range(N):
            c.rz(theta[theta_t].detach().cpu().item(), ind1)
            theta_t+=1
        for ind1 in range(N-1):
            c.cx(ind1, ind1+1)
        if isRing:
            c.cx(N-1,0)  # Add a CX to make it more complex
    sv = Statevector(c).expectation_value(h)
    return sv.real
def get_opt_cut_sv(theta,N,D,h, isRing):
    c = QuantumCircuit(N)
    t = 0
    for i in range(D):
        for j in range(N):
            c.rx(theta[t].detach().cpu().item(), j)
            t+=1
        for j in range(N):
            c.rz(theta[t].detach().cpu().item(), j)
            t+=1
        for j in range(N-1):
            c.cx(j, j+1)
        if isRing:
            c.cx(N-1,0)  # Add a CX to make it more complex
    sv = Statevector(c)
    my_dict = (sv.probabilities_dict())
    max_key = min(my_dict, key=my_dict.get)
    #print("min key" , max_key)
    probabilities = np.abs(sv) ** 2
    # Step 2: Get index of highest probabili
    max_index = probabilities.argmax()
    return format(max_index,f'0{N}b')

# Custom PyTorch autograd function
class QuantumFunction(Function):
    @staticmethod
    def forward(ctx, theta, gates, N, h, D, isRing):
        ctx.save_for_backward(theta)
        ctx.gates = gates
        ctx.N = N
        ctx.D = D
        ctx.h = h
        ctx.isRing = isRing
        output = run_circuit_sv(theta.detach(), N,D,h, isRing)
        return torch.tensor([output], dtype=torch.float32)
    @staticmethod
    def backward(ctx, grad_output):
        theta, = ctx.saved_tensors
        grad = parameter_shift_grad_sv(theta.detach(), ctx.gates, ctx.N, ctx.D, ctx.h, ctx.isRing)
        return grad_output *grad, None, None, None, None, None
# Parameter-shift gradient manually
def parameter_shift_grad_sv(params, gates, N, D, h, isRing):
    gradients = np.zeros_like(params)
    shift = torch.tensor(torch.pi / 2)
    for i in range(len(params)):
        params_forward = params.clone()
        params_backward = params.clone()
        params_forward[i] += shift
        params_backward[i] -= shift
        gradients[i] = (run_circuit_sv(params_forward,N,D,h, isRing) - run_circuit_sv(params_backward, N,D,h, isRing)) / 2
    return gradients

# Wrap into a PyTorch module
class QuantumCircuitTorch(torch.nn.Module):
    def __init__(self, theta, N, D, h, isRing=True):
        super().__init__()
        self.N = N
        self.D = D
        self.h = h
        self.gates = []
        self.isRing = isRing
        self.theta = torch.nn.Parameter(torch.tensor(theta, requires_grad=False))

    def forward(self):
        return QuantumFunction.apply(self.theta, self.gates, self.N, self.h, self.D, self.isRing)


In [ ]:
# =========================
# Resume-in-place utilities
# =========================
import os, csv, json
import torch

def _read_summary(summary_csv="bench_10S_10I/summary.csv"):
    with open(summary_csv, "r") as f:
        return list(csv.DictReader(f))

def _write_summary_header_if_missing(summary_csv="bench_10S_10I/summary.csv"):
    if not os.path.exists(summary_csv):
        with open(summary_csv, "w", newline="") as f:
            csv.writer(f).writerow(["run_id","method","N","depth","seed","theta_path","iter_csv"])

def _find_run_row(method: str, N: int, D: int, seed: int, summary_csv="bench_10S_10I/summary.csv"):
    """Return the LAST matching summary row (dict) or None."""
    if not os.path.exists(summary_csv):
        return None
    rows = _read_summary(summary_csv)
    cand = [r for r in rows if r["method"] == method and int(r["N"]) == N and int(r["depth"]) == D  and int(r["seed"]) == seed]
    return cand[-1] if cand else None

def _read_last_iter(iter_csv_path: str) -> int:
    """Return last iteration index in iter CSV, or -1 if file empty."""
    if not os.path.exists(iter_csv_path):
        return -1
    last = -1
    with open(iter_csv_path, "r") as f:
        r = csv.DictReader(f)
        for row in r:
            last = int(row["iter"])
    return last

def append_run_inplace(
    runlog: RunLog,
    *,
    method: str,
    N: int,
    D: int,
    seed: int,
    outdir: str = "bench_10S_10I",
    summary_csv: str = "bench_10S_10I/summary.csv",
):
    """
    Append new iterations to the *existing* run for (method, N, seed),
    and overwrite its final theta file. If not found, creates a new one.
    Returns the run_id and paths.
    """
    os.makedirs(outdir, exist_ok=True)
    _write_summary_header_if_missing(summary_csv)

    row = _find_run_row(method, N, D, seed, summary_csv)
    if row is None:
        # create a fresh entry (first time)
        import uuid
        run_id = str(uuid.uuid4())[:8]
        theta_path = os.path.join(outdir, f"theta_{run_id}.pt")
        iter_csv_path = os.path.join(outdir, f"iter_{run_id}.csv")
        # write header for new iter file
        with open(iter_csv_path, "w", newline="") as f:
            w = csv.writer(f); w.writerow(["iter","exp_value","best_result","iter_time"])
        # append summary row
        with open(summary_csv, "a", newline="") as f:
            w = csv.writer(f)
            w.writerow([run_id, method, N, D, seed, theta_path, iter_csv_path])
    else:
        run_id      = row["run_id"]
        theta_path  = row["theta_path"]
        iter_csv_path = row["iter_csv"]

    # figure out starting iteration index to continue
    start_iter = _read_last_iter(iter_csv_path) + 1

    # append new iteration rows with offset
    with open(iter_csv_path, "a", newline="") as f:
        w = csv.writer(f)
        for i, (e, b, t) in enumerate(zip(runlog.exp_values, runlog.best_results, runlog.iter_times)):
            w.writerow([start_iter + i, e, json.dumps(b), t])

    # overwrite final theta
    torch.save(runlog.final_theta.detach().cpu(), theta_path)

    return {"run_id": run_id, "theta_path": theta_path, "iter_csv": iter_csv_path, "start_iter": start_iter}

# =========================
# Resume runner (in-place)
# =========================
def benchmark_resume_inplace(
    method: str,
    edges,
    N: int,
    D: int,
    extra_iter_steps: int,
    *,
    seed: int,
    theta_fallback: torch.Tensor,   # used only if no prior run exists
    rank: int = 10,
    isRing: bool = True,
    lr_sv: float = 1e-2,
    lr_tr: float = 1e-2,
    shots: int | None = None,
    batch_size_ef: int = 200,
    batch_size_cs: int = 300,
    batch_size_ps: int = 200,
    outdir: str = "bench_10S_10I",
):
    """
    Continue training the SAME run (method, N, seed). If not existing, starts fresh.
    Appends new iterations to the existing CSV and overwrites final theta.
    """
    # set seeds
    torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

    # try loading last theta (if run exists)
    row = _find_run_row(method, N, D, seed, os.path.join(outdir, "summary.csv"))
    if row is not None and os.path.exists(row["theta_path"]):
        theta0 = torch.load(row["theta_path"], map_location="cpu").detach()
        print(f"[resume] Loaded previous theta for ({method}, N={N}, seed={seed}).")
    else:
        theta0 = theta_fallback.clone().detach()
        print(f"[resume] No previous run found; starting fresh for ({method}, N={N}, seed={seed}).")

    # build H and circuit (your helpers)
    hm   = create_hamiltonian(N, edges)
    h_qsv = make_hamiltonian(hm)
    h_tr  = build_qiskit_like_hamiltonian(hm)
    circuit = build_circuit(N, D, rank=rank, device="cuda", isRing=isRing)

    mm = {
        "tr_efficient": MeasureMethod.EFFICIENT_CONTRACTION,
        "tr_correct_sampling": MeasureMethod.CORRECT_SAMPLING,
        "tr_sampling": MeasureMethod.SAMPLING,
        "sv" : "SV"
    }[method]
    if mm == MeasureMethod.CORRECT_SAMPLING:
      grad_common = dict(batches=batch_size_cs, shots=(shots or 10_000), D=D, isRing=isRing)
      runner = lambda: run_tr_vqe(mm, circuit, theta0, h_tr, extra_iter_steps, lr_tr, grad_common)
    elif mm == MeasureMethod.SAMPLING:
      grad_common = dict(batches=batch_size_ps, shots=(shots or 10_000), D=D, isRing=isRing)
      runner = lambda: run_tr_vqe(mm, circuit, theta0, h_tr, extra_iter_steps, lr_tr, grad_common)

    elif mm == MeasureMethod.EFFICIENT_CONTRACTION:
      grad_common = dict(batches=batch_size_ef, shots=(shots or 10_000), D=D, isRing=isRing)
      runner = lambda: run_tr_vqe(mm, circuit, theta0, h_tr, extra_iter_steps, lr_tr, grad_common)
    elif mm == "SV":
      # run_statevector_vqe(N: int, D: int, h, iter_steps: int, lr: float = 1e-2, theta0=None, isRing=True)
      runner = lambda: run_statevector_vqe(N=N, D=D, h=h_qsv, iter_steps=extra_iter_steps, lr=lr_sv, theta0=theta0, isRing=isRing)
    else:
      raise ValueError(f"Unknown method: {method}")




    # run extra steps
    log: RunLog = runner()

    # append-in-place + overwrite theta
    info = append_run_inplace(log, method=method, N=N, D=D, seed=seed, outdir=outdir)
    print(f"[resume] Appended {len(log.exp_values)} iters starting from {info['start_iter']}.")
    return info, log


In [ ]:
# =========================
# Initial-theta expectations (no VQE)
# =========================
import csv, os, time
from typing import Optional, Dict

# Map your friendly names to the MeasureMethod enum and a label used in CSV
_TR_METHOD_MAP = {
    "tr_expectation": "EFFICIENT_CONTRACTION",   # your MeasureMethod.EFFICIENT_CONTRACTION
    "tr_sampling": "SAMPLING",                   # your MeasureMethod.SAMPLING
    "tr_correct_sampling": "CORRECT_SAMPLING",   # your MeasureMethod.CORRECT_SAMPLING
}

def _resolve_measure_method(name: str):
    """
    Turns 'EFFICIENT_CONTRACTION' -> MeasureMethod.EFFICIENT_CONTRACTION, etc.
    Falls back gracefully if the symbol is organized differently in your repo.
    """
    try:
        from optimization.gradients.measure import MeasureMethod
    except Exception:
        try:
            # sometimes you re-export the enum elsewhere
            from optimization.gradients.batch_parameter_shift import MeasureMethod
        except Exception:
            MeasureMethod = None

    if MeasureMethod is None:
        raise RuntimeError("Could not import MeasureMethod from your TR code. "
                           "Point _resolve_measure_method to the correct enum import.")

    name = name.upper()
    if not hasattr(MeasureMethod, name):
        raise ValueError(f"MeasureMethod has no attribute '{name}'. "
                         f"Available: {', '.join(a for a in dir(MeasureMethod) if a.isupper())}")
    return getattr(MeasureMethod, name), MeasureMethod


def _eval_tr_once(circuit, theta: torch.Tensor, h_tr, *, mm_name: str,
                  shots: Optional[int]=None, batches: Optional[int]=None, D: int=1, isRing: bool=True) -> float:
    """
    Try a few likely entry points in your codebase to evaluate a single expectation value
    at θ without doing any optimization or gradients.
    1) circuit.expectation(theta, h_tr, method=..., shots=..., batches=...)
    2) measure.expectation(circuit, theta, h_tr) via your BatchParameterShiftGradient holder
    3) a minimal 'measurement runner' adapter if you have one
    If none match your repo, swap the body to your canonical evaluator.
    """
    # Ensure correct dtype/device
    theta = theta.detach()

    # 0) Best-effort: a direct circuit method if your Circuit implements it
    if hasattr(circuit, "expectation"):
        try:
            MM, _ = _resolve_measure_method(mm_name)
            val = circuit.expectation(theta, h_tr, method=MM, shots=shots, batches=batches)
            return float(val.real) if hasattr(val, "real") else float(val)
        except Exception:
            pass  # fall through

    # 1) Use your BatchParameterShiftGradient just as a convenient holder of Measure/Runner
    try:
        from optimization.gradients.batch_parameter_shift import BatchParameterShiftGradient
        MM, _ = _resolve_measure_method(mm_name)
        # BatchParameterShiftGradient(step, batches, shots, method, D)
        grad = BatchParameterShiftGradient(
            torch.pi/2, batches or 1, shots or 0, MM, D
        )
        # Common pattern: grad.measure.expectation(...)
        if hasattr(grad, "measure") and hasattr(grad.measure, "expectation"):
            val = grad.measure.expectation(circuit, theta, h_tr, isRing=isRing)
            return float(val.real) if hasattr(val, "real") else float(val)
    except Exception:
        pass

    # 2) If you have a separate measurement runner (adjust import/name if needed)
    try:
        from optimization.measurement import expectation_value as tr_expectation_value
        MM, _ = _resolve_measure_method(mm_name)
        val = tr_expectation_value(circuit, theta, h_tr, method=MM, shots=shots, batches=batches, isRing=isRing)
        return float(val.real) if hasattr(val, "real") else float(val)
    except Exception:
        pass

    # If we get here, we didn’t find your evaluator — please wire one of the above to your actual call.
    raise RuntimeError(
        f"Could not evaluate TR expectation with method '{mm_name}'. "
        f"Wire _eval_tr_once(...) to your repo's single-eval function."
    )


def compute_initial_expvals_for_methods(
    *,
    edges,              # list of ((u,v), w)
    N: int,
    D: int,
    theta0: torch.Tensor,
    seed: int,
    isRing: bool = True,
    tr_rank: int = 10,
    sv_enabled: bool = True,
    methods: Dict[str, bool] = None,  # e.g., {"sv": True, "tr_sampling": True, ...}
    shots_sampling: int = 10_000,
    batches_sampling: int = 200,
) -> Dict[str, float]:
    """
    Returns a dict: {method_name: exp_value_at_theta0}
    methods: toggle which ones you want. Keys allowed: 'sv', 'tr_sampling', 'tr_correct_sampling', 'tr_expectation'
    """
    if methods is None:
        methods = {"sv": True, "tr_sampling": True, "tr_correct_sampling": True, "tr_expectation": True}

    # Build Hamiltonians once
    hm    = create_hamiltonian(N, edges)
    h_qsv = make_hamiltonian(hm)              # Qiskit SparsePauliOp for statevector path
    h_tr  = build_qiskit_like_hamiltonian(hm) # your internal Hamiltonian object for TR path

    # Build circuit once (TR)
    circuit = build_circuit(N, D, rank=tr_rank, device="cuda", isRing=isRing)

    out: Dict[str, float] = {}

    # 1) Statevector (Qiskit)
    if methods.get("sv", False) and sv_enabled:
        t0 = time.perf_counter()
        sv_val = run_circuit_sv(theta0, N, D, h_qsv, isRing=isRing)
        out["sv"] = float(sv_val)
        _ = time.perf_counter() - t0

    # 2) TR backends (no optimization — single eval)
    for friendly, mm_name in _TR_METHOD_MAP.items():
        if not methods.get(friendly, False):
            continue
        val = _eval_tr_once(
            circuit, theta0, h_tr,
            mm_name=mm_name,
            shots=shots_sampling if "SAMPLING" in mm_name else None,
            batches=batches_sampling if "SAMPLING" in mm_name else None,
            D=D, isRing=isRing
        )
        out[friendly] = float(val)

    return out


def write_init_expvals_csv(rows, csv_path="init_expvals.csv"):
    """
    rows: iterable of dicts with the following keys:
      ['graph_id','N','depth','seed','method','exp_value']
    """
    os.makedirs(os.path.dirname(csv_path) or ".", exist_ok=True)
    need_header = not os.path.exists(csv_path)
    with open(csv_path, "a", newline="") as f:
        w = csv.DictWriter(f, fieldnames=["graph_id","N","depth","seed","method","exp_value"])
        if need_header:
            w.writeheader()
        for r in rows:
            w.writerow(r)


In [ ]:
# =========================
# Initial-theta expectations (no VQE)
# =========================
import csv, os, time
from typing import Optional, Dict

# Map your friendly names to the MeasureMethod enum and a label used in CSV
_TR_METHOD_MAP = {
    "tr_expectation": "EFFICIENT_CONTRACTION",   # your MeasureMethod.EFFICIENT_CONTRACTION
    "tr_sampling": "SAMPLING",                   # your MeasureMethod.SAMPLING
    "tr_correct_sampling": "CORRECT_SAMPLING",   # your MeasureMethod.CORRECT_SAMPLING
}

def _resolve_measure_method(name: str):
    """
    Turns 'EFFICIENT_CONTRACTION' -> MeasureMethod.EFFICIENT_CONTRACTION, etc.
    Falls back gracefully if the symbol is organized differently in your repo.
    """
    try:
        from measure.enums import MeasureMethod
    except Exception:
        try:
            # sometimes you re-export the enum elsewhere
            from optimization.gradients.batch_parameter_shift import MeasureMethod
        except Exception:
            MeasureMethod = None

    if MeasureMethod is None:
        raise RuntimeError("Could not import MeasureMethod from your TR code. "
                           "Point _resolve_measure_method to the correct enum import.")

    name = name.upper()
    if not hasattr(MeasureMethod, name):
        raise ValueError(f"MeasureMethod has no attribute '{name}'. "
                         f"Available: {', '.join(a for a in dir(MeasureMethod) if a.isupper())}")
    return getattr(MeasureMethod, name), MeasureMethod


def _eval_tr_once(circuit, theta: torch.Tensor, h_tr, *, mm_name: str,
                  shots: Optional[int]=None, batches: Optional[int]=None, D: int=1, isRing: bool=True) -> float:
    """
    Robust fallback: route through your minimize() stack for a single, no-op 'iteration'
    (lr=0, steps=1). That forces one evaluation using the exact MeasureMethod backend,
    but will not update theta.
    """
    return __eval_tr_via_minimize_single_eval(
        circuit, theta, h_tr,
        mm_name=mm_name,
        shots=shots, batches=batches, D=D, isRing=isRing
    )


def __eval_tr_via_minimize_single_eval(circuit, theta, h_tr, *, mm_name: str,
                                       shots: Optional[int], batches: Optional[int],
                                       D: int, isRing: bool) -> float:
    """
    Uses your BatchParameterShiftGradient + Optimizer + minimize(...) pipeline,
    but with lr=0 and steps=1, to get a single expectation at the *initial* theta.
    """
    # # Imports from your repo (same ones you use in run_tr_vqe)
    from optimization.gradients.batch_parameter_shift import BatchParameterShiftGradient
    from optimization.optimizer import Optimizer
    # from measure.enums import MeasureMethod # adjust if your enum lives elsewhere
    # from optimization.minimize import minimize                  # adjust to your actual module name

    # Resolve MeasureMethod enum value from string
    mm_name = mm_name.upper()
    if not hasattr(MeasureMethod, mm_name):
        raise ValueError(f"MeasureMethod has no attr '{mm_name}'")
    MM = getattr(MeasureMethod, mm_name)

    # Build gradient object (shots/batches only matter for sampling-based methods)
    grad = BatchParameterShiftGradient(
        torch.pi/2,
        int(batches or 1),
        int(shots or 0),
        MM,
        D
    )

    # Optimizer with lr=0 so parameters will not change
    opt = Optimizer(torch.optim.AdamW, {'lr': 0.0})

    # Make sure theta requires no grad (we just want the forward value)
    theta0 = theta.detach().clone().requires_grad_(False)

    # Call your minimize() for a *single* step; most implementations record
    # the initial eval, then (optionally) step. With lr=0, theta stays the same.
    new_theta, exp_vals, best_results, iter_times = minimize(
        circuit, theta0, h_tr,
        opt, grad,
        1,  # steps
        best_value_method=None   # not needed, we just want exp_vals[0]
        #isRing=isRing
    )

    if len(exp_vals) == 0:
        raise RuntimeError("minimize() returned no exp_vals; ensure it logs the initial evaluation.")

    # Return the very first evaluation (at initial theta)
    v0 = exp_vals[0]
    return float(v0.real) if hasattr(v0, "real") else float(v0)


def write_init_expvals_csv(rows, csv_path="init_expvals.csv"):
    """
    rows: iterable of dicts with the following keys:
      ['graph_id','N','depth','seed','method','exp_value']
    """
    os.makedirs(os.path.dirname(csv_path) or ".", exist_ok=True)
    need_header = not os.path.exists(csv_path)
    with open(csv_path, "a", newline="") as f:
        w = csv.DictWriter(f, fieldnames=["graph_id","N","depth","seed","method","exp_value"])
        if need_header:
            w.writeheader()
        for r in rows:
            w.writerow(r)


In [ ]:
# Load initial parameters
theta_init_by_seed = []
for i in range(10):
  a = torch.load(f"/content/drive/MyDrive/TRVQA/benchmark/theta_inits/theta_init_s{i}.pt")
  b = torch.load(f"/content/drive/MyDrive/TRVQA/benchmark/theta_inits/theta_init_s{i}_extra.pt")
  theta_init_by_seed.append(torch.cat([a,b]))


In [ ]:
with open("benchmark/experiment_grid.json", "r") as f:
    loaded = json.load(f)

# Convert lists back to tuples if you want
def convert_lists_to_tuples(obj):
    if isinstance(obj, list):
        # Check if list looks like an edge [(0,1), w]
        if len(obj) == 2 and isinstance(obj[0], list) and isinstance(obj[1], (int, float)):
            return (tuple(obj[0]), obj[1])
        else:
            return [convert_lists_to_tuples(x) for x in obj]
    if isinstance(obj, dict):
        return {k: convert_lists_to_tuples(v) for k, v in obj.items()}
    return obj

graph_by_seed = [convert_lists_to_tuples(entry) for entry in loaded]

In [ ]:
N_list[:1]

[4]

In [ ]:

# =========================
# Driver: collect ONLY initial-theta expectations (no training)
# =========================

# Configure which methods to record
method_switch = {
    "sv": True,
    "tr_sampling": True,
    "tr_correct_sampling": True,
    "tr_expectation": True,
}

out_csv = "ex_value_pots/init_expvals_full.csv"   # choose your preferred path
graph_lookup = {(g['N'], g['graph_id']): g['G'] for g in graph_by_seed}
for N in [16,24]:
  for seed in range(10):                         # or range(10)
          for D in [1,2,3,4,5,6,7,8,9]:
              param_size = 2 * D * N
              theta0 = theta_init_by_seed[seed][:param_size].detach().clone()
              g = graph_lookup[(N,seed)]
              # Compute all requested methods at θ0
              expvals = compute_initial_expvals_for_methods(
                  edges=g,
                  N=N,
                  D=D,
                  theta0=theta0,
                  seed=seed,
                  isRing=True,
                  tr_rank=10,
                  sv_enabled=True,
                  methods=method_switch,
                  shots_sampling=10_000,
                  batches_sampling=200,  # tweak as you like
              )

              # Write a row per method
              rows = []
              for meth, val in expvals.items():
                  rows.append({
                      "graph_id": f"N{N}_S{seed}",   # use index or some hash; customize if needed
                      "N": N,
                      "depth": D,
                      "seed": seed,
                      "method": meth,
                      "exp_value": val,
                  })
              write_init_expvals_csv(rows, csv_path=out_csv)
              print(f"[init-exp] seed={seed} graph={N} N={N} D={D} -> {expvals}")


Progress: [>                             ] 0% | ETA: 00:00 | Loss: 79.3252[init-exp] seed=0 graph=16 N=16 D=1 -> {'sv': 79.19934433941022, 'tr_expectation': 79.19856262207031, 'tr_sampling': 75.4209976196289, 'tr_correct_sampling': 79.32520000000001}
Progress: [>                             ] 0% | ETA: 00:00 | Loss: 72.0405[init-exp] seed=0 graph=16 N=16 D=2 -> {'sv': 72.2469323377681, 'tr_expectation': 72.24671173095703, 'tr_sampling': 72.4939956665039, 'tr_correct_sampling': 72.04050000000001}
Progress: [>                             ] 0% | ETA: 00:00 | Loss: 75.1329[init-exp] seed=0 graph=16 N=16 D=3 -> {'sv': 75.07940747711972, 'tr_expectation': 74.80183410644531, 'tr_sampling': 74.8781967163086, 'tr_correct_sampling': 75.1329}
Progress: [>                             ] 0% | ETA: 00:00 | Loss: 75.6644[init-exp] seed=0 graph=16 N=16 D=4 -> {'sv': 76.0027297143659, 'tr_expectation': 58.07188034057617, 'tr_sampling': 75.52189636230469, 'tr_correct_sampling': 75.6644}
Progress: [>     

In [ ]:
torch.cuda.empty_cache()